## Introduction

In this exercise you will train and evaluate a decision tree. Opposed to the previous exercises with the topics _univariate linear regression_, _multivariate linear regression_, _logistic regression_ and _bias variance tradeoff_, you will not implement the algorithms from scratch using numpy. Instead you will use two python packages written on top of numpy, namely the _pandas_ and the _scikit-learn_ package, which are both widely used by the machine learning community. The steps can be broken down into:

1. Load the data from a _csv file_ into a pandas DataFrame object
2. Preprocess the data
3. Train a decision tree and visualize it.
4. Do hyperparameter optimization by dividing the dataset into a fixed training set and a fixed validation set.
5. Do hyperparameter optimization by crossvalidation
6. Do hyperparameter optimization by gridsearch

Afterwards, show that you understand, what computations are done by the _scikit-learn_ package you have used by manually computing:

7. The entropy for one node
8. The information gain for one node

**Note:**

As this exercise heavily focuses on the usage of the high-level APIs _pandas_ and _scikit-learn_, reading their documentations (just the parts corresponding to the current task) is strongly recommended:

- https://scikit-learn.org/stable/documentation.html#
- https://pandas.pydata.org/pandas-docs/stable/search.html?q=&check_keywords=yes&area=default

In [1]:
import numpy as np
import pandas as pd

#from sklearn.preprocessing import Imputer # in newer versions: 
from sklearn.impute import SimpleImputer as Imputer

from sklearn import tree

#old: from sklearn.externals.six import StringIO
from six import StringIO

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from graphviz import Source

ModuleNotFoundError: No module named 'graphviz'

## Teaching Content

### Decision Trees for Classification

In a decision tree the data is split at each node according to a decision rule. This corresponds to nested *if-then-else*-rules. In the *if*-part of such a rule are decision is made based on a feature of the data record. 

We will use the scikit learn implementation. For this implementation the features must be binary or have (at least) ordinal characteristic. If a feature is e.g. nominal with many values, it must be converted to a set of binary (one-hot-coded) features.


The splitting rules in the scikit learn implementation are binary and are based on a threshold, e.g.
  - _if $x_6 <= 2.14$ then_ left subbranch, *else* right subbranch.         
  - binary features must be coded as 0/1, so the rule becomes: if $x_2 <= 0.5$ _then_ left subbranch, _else_ right subbranch. 


<!--The structure of the tree will be determined by data (see below) and a training procedure.-->

In the leaves of the tree the (class) predictions are made. There are two possibilities for such an inference: 
   - hard assignment: Predict for the data records which end up on a leaf by the majority class of the training data that end up on that leaf.          
   - soft assignment: Assign probabilities according to the distribution of the classes in respect to the training data which end up on that leaf.

As an example of a decision tree we will learn the following tree from the titanic data set: 

<img src="https://gitlab.com/deep.TEACHING/educational-materials/raw/master/media/klaus/exercise-decision-trees-a-tree.png" width="1024" alt="internet connection needed">

A full explanation of the tree will be given later. Here just look at the decision rules (first line of the inner nodes) and at the last line of the leafs. In each leaf you see an array (values) with counts of the different targets for the train data: [number_died, number_survivors] .

### Learning 

Finding a tree that splits the training data optimal is [np-hard](https://en.wikipedia.org/wiki/NP-hardness). Therefore often a *greedy*-strategy is used:

To build up a decision tree the algorithm starts at the root of the tree. The feature and the threshold 
that splits the training data best (with respect to the classes) are chosen. In an iterative way the whole tree is build up by such splitting rules. 

There are different criteria for measuring the "separation (split) quality". The most important ones are:

- Gini Impurity 
- Information Gain 

In this tutorial we concentrate on the information gain.

### Information Gain as Splitting Criterion

The entropy with respect to the target class variable $y$ of a training data set $\mathcal D$ is defined as:

$$
 H(y, \mathcal D) = - \sum_{y \in \mathcal Y} p(y|\mathcal D) \log_2 p(y|\mathcal D)
$$
with the domain of the target values $\mathcal Y = \{t_1, t_2,... \}$.


The probabilities are estimated by 
$$
  p(y=t_i, \mathcal D) = |\mathcal D^{(y=t_i)}| /|\mathcal D| 
$$    


with the number of training data $|\mathcal D|$  and the number of training data $|\mathcal D^{(y=t_i)}|$ with target label $t_i$: 


On a node a (binary) split on a feature $x_k$ is made by the split rule $x_k \leq v$. 
As result there are two data sets $\mathcal D_0$ and $\mathcal D_1$ for the left resp. the right branch.

The feature $x_k$ and the split value $v$ are chosen that they maximize the 'reduction of the entropy' measured by the information gain $I$:
$$
  I(y; x_k) = H(y, \mathcal D) - H(y|x_k) = H(y, \mathcal D) - \sum_{j=0}^1 p_jH(y, \mathcal D_j) =
  H(y, \mathcal D) + \sum_{j=0}^1  \sum_{y \in \mathcal Y} \frac{|\mathcal D_j|}{|\mathcal D|} p(y|\mathcal D_j) \log_2 p(y|\mathcal D_j)
$$
Note that $p_{j=0}$  is the estimated probability that a random data record of $\mathcal D$ has feature value $x_k \leq v$ which can be estimated by ${|\mathcal D_0|}/{|\mathcal D|}$ (analog for $j=1$).

$p(y=t_i|\mathcal D_0)$ can also be estimated by the fraction of the counts ${|\mathcal D_0^{(y=t_i)}|}/{|\mathcal D_0|}$. 
So the information gain can be computed just with counts:


$$
  I(y; x_k) = -
   \sum_{y \in \mathcal Y} \frac{|\mathcal D^{(y=t_i)}|}{|\mathcal D|}  \log_2 \frac{|\mathcal D^{(y=t_i)}|}{|\mathcal D|} + \sum_{j=0}^1  \sum_{y \in \mathcal Y} \frac{|\mathcal D_j^{(y=t_i)}|}{|\mathcal D|}  \log_2 \frac{|\mathcal D_j^{(y=t_i)}|}{|\mathcal D_j|}
$$


<!--$|\mathcal D_0|$ respectivly $|\mathcal D_1|$ is the number of elements in the splitted data sets.-->
 

### Overfitting

Deep decision trees generalize often poorly. The following remedies reduce overfitting: 

- Limitation of the maximal depth of the tree. 
- Pruning with an validation set either during training (pre-pruning) or after training (post-pruning).
- Dimensionality reduction (reducing the number of features before training)


Also often combining decision trees to an ensemble (decision forests) is used against overfitting.  

### Loading the Data

First we read in the _titanic data set_ with _pandas_

**Task:**

1. Complete the code to load the dataset as [_pandas_](http://pandas.pydata.org/) [DataFrame](http://pandas.pydata.org/pandas-docs/stable/dsintro.html) object.
2. Either download the [_titanic-train.csv_](https://gitlab.com/deep.TEACHING/educational-materials/raw/master/datasets/titanic-train.csv) or alternatively read the url directly into a pandas DataFrame.

In [2]:
### Exercise: Load the csv file as pandas DataFrame
url = 'https://gitlab.com/deep.TEACHING/educational-materials/raw/master/notebooks/data/titanic-train.csv'

train_df = pd.read_csv(url)

In [3]:
### Not an exercise, just execute the cell

print(train_df.ndim)
print(train_df.shape)

2
(891, 12)


In [4]:
train_df


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [5]:
train_df["Sex"]

0        male
1      female
2      female
3      female
4        male
        ...  
886      male
887    female
888    female
889      male
890      male
Name: Sex, Length: 891, dtype: object

In [6]:
train_df["Sex"] = train_df["Sex"].apply(lambda sex: 1 if sex == 'female' else 0)
# or
#df.Sex[df["Sex"]=='female'] = 1
#df.Sex[df["Sex"]=='male'] = 0

In [7]:
y = targets = labels = train_df["Survived"].values

columns = ["Fare", "Pclass", "Sex", "Age", "SibSp"]
x = train_df[list(columns)].values
x

array([[ 7.25  ,  3.    ,  0.    , 22.    ,  1.    ],
       [71.2833,  1.    ,  1.    , 38.    ,  1.    ],
       [ 7.925 ,  3.    ,  1.    , 26.    ,  0.    ],
       ...,
       [23.45  ,  3.    ,  1.    ,     nan,  1.    ],
       [30.    ,  1.    ,  0.    , 26.    ,  0.    ],
       [ 7.75  ,  3.    ,  0.    , 32.    ,  0.    ]])

In [8]:
print("-----------First 5 with nan BEFORE----------")
nanMask = np.argwhere(np.isnan(x))
print(x[nanMask[0:5,0]])

-----------First 5 with nan BEFORE----------
[[ 8.4583  3.      0.         nan  0.    ]
 [13.      2.      0.         nan  0.    ]
 [ 7.225   3.      1.         nan  0.    ]
 [ 7.225   3.      0.         nan  0.    ]
 [ 7.8792  3.      1.         nan  0.    ]]


In [9]:
print("-----------First 5 with nan AFTER----------")
print(x[nanMask[0:5,0]])

-----------First 5 with nan AFTER----------
[[ 8.4583  3.      0.         nan  0.    ]
 [13.      2.      0.         nan  0.    ]
 [ 7.225   3.      1.         nan  0.    ]
 [ 7.225   3.      0.         nan  0.    ]
 [ 7.8792  3.      1.         nan  0.    ]]


In [10]:
print("-----------First 5 with nan BEFORE----------")
nanMask = np.argwhere(np.isnan(x))
print(x[nanMask[0:5,0]])

imp = Imputer()#missing_values=nan, strategy='mean') #, axis=0)
x = imp.fit_transform(x)

print("-----------First 5 with nan AFTER----------")
print(x[nanMask[0:5,0]])

-----------First 5 with nan BEFORE----------
[[ 8.4583  3.      0.         nan  0.    ]
 [13.      2.      0.         nan  0.    ]
 [ 7.225   3.      1.         nan  0.    ]
 [ 7.225   3.      0.         nan  0.    ]
 [ 7.8792  3.      1.         nan  0.    ]]
-----------First 5 with nan AFTER----------
[[ 8.4583      3.          0.         29.69911765  0.        ]
 [13.          2.          0.         29.69911765  0.        ]
 [ 7.225       3.          1.         29.69911765  0.        ]
 [ 7.225       3.          0.         29.69911765  0.        ]
 [ 7.8792      3.          1.         29.69911765  0.        ]]


In [11]:
clf = tree.DecisionTreeClassifier(criterion="entropy", max_depth=3)
clf = clf.fit(x, y)

In [12]:
graph = Source(tree.export_graphviz(clf, out_file=None
   , feature_names=columns, class_names=['died', 'survived'] 
   , filled = True))
graph

NameError: name 'Source' is not defined

In [13]:
### To open in seperate window and save it
graph.format = 'png'
graph.render('dtree_render',view=True)

NameError: name 'graph' is not defined

In [14]:
length = int(len(x) * 0.75)
print(length)
x_train = x[:length]
x_val = x[length:]
y_train = y[:length]
y_val = y[length:]

668


In [15]:
for depth in range(1,20):
    clf = tree.DecisionTreeClassifier(criterion="entropy", max_depth=depth)
    clf = clf.fit(x_train, y_train)
    preds = clf.predict(x_val)
    acc = (preds - y_val) 
    acc = [1 if i == -1 else i for i in acc]
    acc = 1 - np.sum(acc) / len(y_val)
    print('depth: ', depth, ' acc: ', acc)

depth:  1  acc:  0.789237668161435
depth:  2  acc:  0.789237668161435
depth:  3  acc:  0.8385650224215246
depth:  4  acc:  0.8340807174887892
depth:  5  acc:  0.8340807174887892
depth:  6  acc:  0.8340807174887892
depth:  7  acc:  0.8385650224215246
depth:  8  acc:  0.8161434977578476
depth:  9  acc:  0.8295964125560538
depth:  10  acc:  0.8251121076233183
depth:  11  acc:  0.8340807174887892
depth:  12  acc:  0.8340807174887892
depth:  13  acc:  0.8161434977578476
depth:  14  acc:  0.8116591928251121
depth:  15  acc:  0.8116591928251121
depth:  16  acc:  0.820627802690583
depth:  17  acc:  0.8071748878923767
depth:  18  acc:  0.8026905829596412
depth:  19  acc:  0.8026905829596412


In [16]:
depths = []
for i in range(1,20):
    clf = tree.DecisionTreeClassifier(criterion="entropy", max_depth=i)
    scores = cross_val_score(estimator=clf, X=x, y=y, cv=10, n_jobs=4, scoring="accuracy")
    depths.append((i, scores.mean()))
    
for d in depths: print('depth: ', d[0], ' acc: ', d[1])

depth:  1  acc:  0.786729088639201
depth:  2  acc:  0.7688764044943821
depth:  3  acc:  0.8170536828963796
depth:  4  acc:  0.8081897627965045
depth:  5  acc:  0.8227590511860174
depth:  6  acc:  0.8182896379525593
depth:  7  acc:  0.8194007490636703
depth:  8  acc:  0.8149812734082398
depth:  9  acc:  0.8182771535580524
depth:  10  acc:  0.8137578027465668
depth:  11  acc:  0.8171161048689137
depth:  12  acc:  0.802521847690387
depth:  13  acc:  0.8148689138576779
depth:  14  acc:  0.7980649188514357
depth:  15  acc:  0.8036953807740325
depth:  16  acc:  0.8014357053682897
depth:  17  acc:  0.7991760299625469
depth:  18  acc:  0.7868539325842696
depth:  19  acc:  0.786816479400749


In [17]:
parameters = {'max_depth':range(1,20), 'criterion':['entropy','gini']}
clf = GridSearchCV(tree.DecisionTreeClassifier(), parameters, n_jobs=4, cv=10)
clf.fit(X=x, y=y)
tree_model = clf.best_estimator_
print (clf.best_score_, clf.best_params_) 

0.8227590511860174 {'criterion': 'entropy', 'max_depth': 5}


The entropy with respect to the target class variable $y$ of a training data set $\mathcal D$ is defined as:

$$
 H(y, \mathcal D) = - \sum_{y \in \mathcal Y} p(y|\mathcal D) \log_2 p(y|\mathcal D)
$$

In [41]:
nb_survived = y.sum()
p_survived = float(y.sum())/len(y)
p_not_survived = 1. - p_survived
entropy_before_split = -p_survived * np.log2(p_survived) - p_not_survived * np.log2(p_not_survived)
z=p_survived+p_not_survived
print(z)
t3 = -z * np.log2(z)
print(t3)
t = - p_survived * np.log2(p_survived)
t2 = - p_not_survived * np.log2(p_not_survived)


1.0
-0.0


**Task 1:**

Recompute the root node entropy. On Pen & paper or with just some lines of code here using basic mathematical operations.

survived = $342$

pSurvived = $\frac{342}{891}$

pNotSurvived = $1-$ pSurvived = $1-0.3838383838383838=0.6161616161616161$

entropyBeforeSplit = $-pSurvived \cdot \log_2(pSurvived) - pNotSurvived \cdot \log_2(pNotSurvived)$

In [29]:
342/891
1-0.3838383838383838


0.6161616161616161

In [ ]:
y